In [ ]:
import json, os, glob
from PIL import Image, ImageDraw
import numpy as np
from PIL.ImageColor import getrgb

In [ ]:
class_dict = {'NFL':13, 'GCL+IPL':5, 'INL':17, 'OPL':2,'ONL':22,'EZ':9,'RPE':3, 'CHR':15}

In [ ]:
json_file = './labels.json'
original_images = './trainingSet/origImages/'
result_save = './trainingSet/maskImages/'

In [ ]:
all_labels = json.load(open(json_file))

In [ ]:
rawlist = all_labels.keys()
print(rawlist)
print(all_labels.keys())

In [ ]:
# def get_data(data, X,Y,L):
def get_data(data):

    X = []; Y = []; L=[] #pre-allocate lists to fill in a for loop
    for k in data['regions']: #cycle through each polygon
        # get the x and y points from the dictionary
        X.append(data['regions'][k]['shape_attributes']['all_points_x'])
        Y.append(data['regions'][k]['shape_attributes']['all_points_y'])
        L.append(data['regions'][k]['region_attributes']['label'])
    return Y,X,L #image coordinates are flipped relative to json coordinates

In [ ]:
def get_mask(X, Y, nx, ny, L, class_dict):
    # get the dimensions of the image
    mask = np.zeros((nx,ny))

    for y,x,l in zip(X,Y,L):
        # the ImageDraw.Draw().polygon function we will use to create the mask
        # requires the x's and y's are interweaved, which is what the following
        # one-liner does
        polygon = np.vstack((x,y)).reshape((-1,),order='F').tolist()

        # create a mask image of the right size and infill according to the polygon
        size = (nx, ny)
        if nx>ny:
           x,y = y,x
           img = Image.new("L",  (nx, ny), 0)
        elif ny>nx:
           img = Image.new("L", (ny,nx), 0)
        else:
           img = Image.new("L", (nx, ny),  0)

        ImageDraw.Draw(img).polygon(polygon, fill = 1, outline = 1)
        # turn into a numpy array
        m = np.flipud(np.rot90(np.array(img)))

        try:
            mask[m==1] = class_dict[l]
        except:
            mask[m.T==1] = class_dict[l]  

         # mask[m==1] = class_dict[l]

    return mask

In [ ]:
def rescale(dat,mn,mx):
    '''
    rescales an input dat between mn and mx
    '''
    m = min(dat.flatten())
    M = max(dat.flatten())
    return (mx-mn)*(dat-m)/(M-m)+mn

In [ ]:
for name in rawlist:
    
    new_path = original_images+name
    image = Image.open(new_path)
    nx, ny, nz = np.shape(image)
    
    X, Y, L = get_data(all_labels[name])

    mask = get_mask(X, Y, nx, ny, L, class_dict)

    mask = Image.fromarray(rescale(mask,0,255)).convert('RGB')
    mask.save(result_save+name.replace('imagery','labels'), format='PNG')


In [ ]:
import cv2

In [ ]:
img = cv2.imread(result_save+'NORMAL1.jpeg')
img = cv2.resize(img,(640,640), interpolation = cv2.INTER_NEAREST)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = img.shape
train_masks_reshaped = img.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)


In [ ]:

train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)
np.unique(train_masks_input)

In [ ]:
np.unique(img)

In [ ]:
#Capture mask/label info as a list
train_masks = [] 
for directory_path in glob.glob(result_save):
    for mask_path in glob.glob(os.path.join(directory_path, "*.jpeg")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (640,640), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        train_masks.append(mask)
        
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

In [ ]:
for i in range(100):
    print(np.unique(train_masks[i]),'  label'+str(i))


In [ ]:
from matplotlib import pyplot as plt

plt.imshow(train_masks[1])

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(train_masks[3])